In [ ]:
import csv
from collections import defaultdict

def decimal_to_dms(decimal_deg):
    sign = 1 if decimal_deg >= 0 else -1
    decimal_deg_abs = abs(decimal_deg)
    degrees = int(decimal_deg_abs)
    remainder = decimal_deg_abs - degrees
    minutes_full = remainder * 60
    minutes = int(minutes_full)
    seconds = round((minutes_full - minutes) * 60)
    # Handle overflow in seconds and minutes
    if seconds >= 60:
        seconds -= 60
        minutes += 1
    if minutes >= 60:
        minutes -= 60
        degrees += 1
    degrees = degrees * sign
    return f"{degrees}'{minutes}'{seconds}"

def csv_to_scenario(csv_filename, output_filename):
    # Read CSV
    with open(csv_filename, 'r') as f:
        reader = csv.DictReader(f)
        rows = list(reader)

    # Group rows by flight id (id column)
    flights = defaultdict(list)
    for row in rows:
        flight_id = row['id']
        flights[flight_id].append(row)

    scenario = []
    scenario.append("00:00:00.00>ASAS OFF")

    for flight_id, rows in flights.items():
        # Sort rows by time
        sorted_rows = sorted(rows, key=lambda r: int(r['time']))

        # Get acid from callsign
        acid = sorted_rows[0]['callsign'].strip()

        # First row is CRE command
        first_row = sorted_rows[0]
        lat = float(first_row['lat'])
        lon = float(first_row['lon'])
        geoaltitude = float(first_row['geoaltitude'])
        heading = float(first_row['heading'])
        speed_km_per_s = float(first_row['speed'])  # Assuming 'speed' column exists

        # Convert speed to knots (1 km/s = 1943.844 knots)
        speed_knots = speed_km_per_s * 1943.844

        # Aircraft type is A320 (default)
        actype = 'A320'

        # CRE command: CRE acid, actype, lat, lon, alt, hdg, spd
        cre_command = f"00:00:00.00>CRE {acid},{actype},{lat},{lon},{geoaltitude},{heading},{speed_knots:.2f}"
        scenario.append(cre_command)

        # SPD and ALT commands based on first row
        spd_command = f"00:00:00.00>SPD {acid} {round(speed_knots)}"
        scenario.append(spd_command)

        alt_command = f"00:00:00.00>ALT {acid} {geoaltitude}"
        scenario.append(alt_command)

        # Add waypoints for remaining rows
        waypoint_rows = sorted_rows[1:]
        first_wp = True
        for row in waypoint_rows:
            lat_wp = float(row['lat'])
            lon_wp = float(row['lon'])
            alt_wp = float(row['geoaltitude'])

            dms_lat = decimal_to_dms(lat_wp)
            dms_lon = decimal_to_dms(lon_wp)

            addwpt_command = f"00:00:00.00>{acid} ADDWPT {dms_lat},{dms_lon}, {alt_wp}"
            scenario.append(addwpt_command)

            # Add AT command to enable VNAV ON for the first waypoint
            if first_wp:
                at_command = f"00:00:00.00>{acid} AT {dms_lat},{dms_lon} STACK {acid} VNAV ON"
                scenario.append(at_command)
                first_wp = False

    # Write scenario to output file
    with open(output_filename, 'w') as f:
        f.write('\n'.join(scenario))

# Example usage:
# csv_to_scenario('input.csv', 'output.scn')